In [1]:
from evasion import *
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

In [7]:
network = NETWORKS[0]
print(network)
paths = network.evasion_paths(compute_homology=False)
print(f"found {len(paths)} paths")

Network "instructions example" with 6 sensors in 8x8 room (period 40)


288

In [10]:
evasion_path = paths[42]
# path

In [11]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 300
plt.ioff()
fig, ax = plt.subplots()

cmap = cm.get_cmap('viridis')
anchor = lambda pos: (pos.x - 1, pos.y - 1)

ax.set_xlim(0, network.room_width)
ax.set_ylim(0, network.room_height)
ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.invert_yaxis()


INTERP_FRAMES = 1
frames_per_step = 1 + INTERP_FRAMES


def init():
    global squares, plotted_pts
    # network.reset()
    squares = [
        plt.Rectangle(anchor(sensor.curr_pos), 2, 2, color=cmap(i / len(network.sensors)))
        for i, sensor in enumerate(network.sensors)
    ]

    # Add the squares to the axes
    for square in squares:
        ax.add_patch(square)

    plotted_pts = []

# Animation update function
def update(t):
    print(t, end=" ")
    global squares, plotted_pts

    for sensor, square in zip(network.sensors, squares):
        square.set_xy(anchor(sensor.curr_pos))
        sensor.move()

    positions_after = evasion_path[t]
    positions_before = evasion_path[t - 1]
    
    for p in plotted_pts:
        p.remove()

    plotted_pts.clear()
    
    for pos in positions_after:
        plotted_pts.extend(ax.plot(pos.x + 0.5, pos.y + 0.5, 'o', color='green'))
    
    for pos in positions_before:
        plotted_pts.extend(ax.plot(pos.x + 0.5, pos.y + 0.5, 'x', color='black'))



matplotlib.animation.FuncAnimation(fig, update, init_func=init, frames=network.period * frames_per_step)

C:\Users\andre\AppData\Local\Temp\ipykernel_20280\4248378858.py:6: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('viridis')


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 